# Year/Season Level Data

In [1]:
# import necessary packages
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pprint import pprint
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

C:\Users\kpokr\AppData\Local\Temp\ipykernel_8528\2024037245.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# grab list of NHL stats files
DATA_FOLDER = '../data/interim/'
FILE_LIST = os.listdir(DATA_FOLDER)
pprint(FILE_LIST)

['abbrev.csv',
 'AwardsCoaches.csv',
 'AwardsPlayers.csv',
 'Coaches.csv',
 'CombinedShutouts.csv',
 'Goalies.csv',
 'GoaliesSC.csv',
 'GoaliesShootout.csv',
 'HOF.csv',
 'Master.csv',
 'Scoring.csv',
 'ScoringSC.csv',
 'ScoringShootout.csv',
 'SeriesPost.csv',
 'Teams.csv',
 'TeamsHalf.csv',
 'TeamSplits.csv',
 'TeamsPost.csv',
 'TeamsSC.csv',
 'TeamVsTeam.csv']


In [49]:
file_num = 19

df = pd.read_csv(DATA_FOLDER + FILE_LIST[file_num])
print('Viewing:', FILE_LIST[file_num])
print('Number of rows:', len(df))
print('Columns in dataset:', df.columns)
df.sample(10)


Viewing: TeamVsTeam.csv
Number of rows: 25602
Columns in dataset: Index(['year', 'lgID', 'tmID', 'oppID', 'W', 'L', 'T', 'OTL'], dtype='object')


,year,lgID,tmID,oppID,W,L,T,OTL
5787,1978,NHL,COR,LAK,0,2,2.0,NaN
23215,2009,NHL,CHI,PHI,0,1,NaN,0.0
12240,1993,NHL,DAL,CAL,3,0,1.0,NaN
23504,2009,NHL,NJD,OTT,3,1,NaN,0.0
18846,2002,NHL,PHI,MTL,4,0,0.0,0.0
17877,2001,NHL,NYI,BUF,1,2,1.0,0.0
8750,1985,NHL,MNS,PHI,0,2,1.0,NaN
4246,1974,WHA,EDO,MFS,3,3,0.0,NaN
19365,2003,NHL,DAL,LAK,2,2,2.0,0.0
11246,1991,NHL,LAK,BUF,1,1,1.0,NaN


**File List:**
- abbrev.csv: reference file defining acronyms used in other data files
- AwardsCoaches.csv: lists awards won by coaches
    - primary key: coachID
- AwardsPlayers.csv: lists awards won by players
    - primary key: playerID (probably not unique)
    - foreign key: lgID
- Coaches.csv: master file of coaches
    - primary key: coachID (probably not unique)
    - foreign keys: lgID, tmID
- CombinedShutouts.csv: lists games where two goals were used to record a shutout
- Goalies.csv: stats for goalies aggregated yearly
- GoaliesSC.csv: stats for goalies during the Stanley Cup finals aggregated yearly
- GoaliesShootout.csv: shootout records for goalies aggregated yearly
- HOF.csv: hall of fame inductees by year
- Master.csv: main dataset with demographic data for players/coaches
- Scoring.csv: player stats aggregated yearly
- ScoringSC.csv: stats for players during the Stanley Cup finals aggregated yearly
- ScoringShootout.csv: shootout records for players aggregatd yearly
- SeriesPost.csv: playoff series statistics aggregated yearly
- Teams.csv: main dataset with team-level stats aggregated yearly
- TeamsHalf.csv: team-level stats aggregated for the first and second halves of each season
- TeamSplits.csv: team-level stats aggregated for each month in a season
- TeamsPost.csv: team-level postseason stats aggregated yearly
- TeamsSC.csv: team-level Stanley Cup finals stats aggregated yearly
- TeamVsTeam.csv: team-level stats in head-to-head match-ups aggregated yearly

In [23]:
# Create df of columns names and dtypes to better determine connections between tables
nhl_data_dict = {'table_name': [],
                 'column_name': [],
                 'data_type': [],
                 'percent_null': []
                 }

for file in FILE_LIST:

    table_name = file.split('.')[0]

    # load in data
    try:
        df = pd.read_csv(DATA_FOLDER + file)
    except:
        df = pd.read_csv(DATA_FOLDER + file, sep='\t')

    num_rows = len(df)
    cols = df.columns

    for col in cols:
        nhl_data_dict['table_name'].append(table_name)
        nhl_data_dict['column_name'].append(col)
        nhl_data_dict['data_type'].append(df.dtypes[col])

        prct_null = round(df[col].isnull().sum()/num_rows, 1)
        nhl_data_dict['percent_null'].append(prct_null * 100)


# convert data dictionary to df
data_df = pd.DataFrame(nhl_data_dict)

data_df.sample(20)


,table_name,column_name,data_type,percent_null
69,GoaliesSC,GA,int64,0.0
237,TeamSplits,DecT,float64,20.0
207,TeamsHalf,W,int64,0.0
160,ScoringShootout,GDG,int64,0.0
177,Teams,franchID,object,0.0
114,Scoring,year,int64,0.0
132,Scoring,PostGP,float64,60.0
174,Teams,year,int64,0.0
25,Coaches,postw,float64,50.0
32,CombinedShutouts,oppID,object,0.0


In [25]:
# isolate all columns with "ID" in name as these are probably what we can use to join data sets
id_fields = data_df[data_df['column_name'].str.contains("ID")]
print(len(id_fields))
id_fields.sort_values(by=['column_name'])

54


,table_name,column_name,data_type,percent_null
34,CombinedShutouts,IDgoalie1,object,0.0
35,CombinedShutouts,IDgoalie2,object,0.0
3,AwardsCoaches,coachID,object,0.0
14,Coaches,coachID,object,0.0
83,Master,coachID,object,90.0
178,Teams,confID,object,40.0
179,Teams,divID,object,20.0
177,Teams,franchID,object,0.0
84,Master,hofID,object,100.0
79,HOF,hofID,object,0.0
